In [9]:
import pandas as pd
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
import numpy as np
import matplotlib.pyplot as plt 
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler

In [10]:
df = pd.read_parquet("../data/static_models/pump3_power_siso.par")
df = df.set_index("time")
df = df.rolling(window=40, center=True).median().dropna()
df = df.resample("5s").mean().dropna()

In [11]:
df['pump3_speed'] = df['pump3_speed'].apply(lambda x: max(0, x))


In [12]:
train_df = df["2024-01-09 10:30:00":"2024-01-09 16:10:00"]
test_df = df["2024-01-10":"2024-02-16 12:00:00"]

In [13]:
from src.system_identification import ARXParametersIdentification
from src.preprocessing import Preprocessor
import pandas as pd
from sysidentpy.basis_function._basis_function import Polynomial

In [14]:
FEATURES = "pump3_speed"
TARGET = 'pump3_power'

sys_id = ARXParametersIdentification(FEATURES, 
                            TARGET, 
                            dataframe= train_df, 
                            model_type= 'siso')

In [15]:
parameters = {
    'estimator': 'ridge_regression',
    'ridge_param': 2e6,
    'order_selection': False,
    'n_terms': 3,
    'extended_least_squares': False,
    'ylag': 2,  
    'xlag': 1, 
    'basis_function': Polynomial(degree=1)
}
    
fitted_model, model_parameters = sys_id.identify_arx_parameters(x_train=train_df[FEATURES], 
                                                                y_train= train_df[TARGET],
                                                                **parameters)
    
model_parameters

/home/alqua/Git/edge_mpc/sys_identification/src/sysidentpy/narmax_base.py:1034: RuntimeWarning: divide by zero encountered in divide
  x = x[1:] / aux_b
/home/alqua/Git/edge_mpc/sys_identification/src/sysidentpy/narmax_base.py:1034: RuntimeWarning: invalid value encountered in divide
  x = x[1:] / aux_b


,Regressors,Parameters,ERR
0,x1(k-1),4.1216E-02,9.9822E-01
1,y(k-1),3.6872E-03,NAN
2,y(k-2),3.1648E-03,NAN


In [16]:
yhat  = sys_id.predict(x_test= test_df[FEATURES], y_test= test_df[TARGET], n_steps_ahead= 200)

#val_plots = sys_id.residuals_analysis(yhat, df[TARGET].values)
#print(model_parameters)

RRSE: 0.08543549246398328
MSE: 3.7078138317809706


In [18]:
test_df["yhat"] = yhat

sys_id.plot_features_and_target(test_df, ['yhat', 'pump3_power'])

FigureWidgetResampler({
    'data': [{'name': '<b style="color:sandybrown">[R]</b> yhat <i style="color:#fc9944">~54m</i>',
              'showlegend': True,
              'type': 'scattergl',
              'uid': '89164da8-f615-42df-8eff-9dd2b6cd1ad8',
              'x': array([datetime.datetime(2024, 1, 10, 0, 0, tzinfo=<UTC>),
                          datetime.datetime(2024, 1, 10, 0, 0, 5, tzinfo=<UTC>),
                          datetime.datetime(2024, 1, 10, 0, 27, 5, tzinfo=<UTC>), ...,
                          datetime.datetime(2024, 2, 16, 9, 42, 55, tzinfo=<UTC>),
                          datetime.datetime(2024, 2, 16, 11, 18, 25, tzinfo=<UTC>),
                          datetime.datetime(2024, 2, 16, 11, 26, 45, tzinfo=<UTC>)], dtype=object),
              'y': array([ 0.        ,  0.        ,  0.        , ..., 41.68785652, 43.57573409,
                          42.33071312])},
             {'name': ('<b style="color:sandybrown">[R' ... 'style="color:#fc9944">~54m</i>'),
